<a href="https://colab.research.google.com/github/bhend449/ML-RTS-detection-model-builder/blob/master/RTS_Detection_Model_alpha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#RTS Detection Modeler

This creates a model to detect bi-stable RTS signals from a large array of unknown signals.

###Mount to Google Drive

Assume the files are stored on a private Google Drive folder

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

###Check location of files 

In [0]:

!ls "/content/gdrive/My Drive/Colab data files"

###Import .mat files with SciPi and prepare the training and testing sets


In [0]:
import matplotlib
import matplotlib.pyplot as plt
import h5py
import numpy as np
import scipy.io
'''
        Import two vectors, one RTS and another white noise
'''

mat = scipy.io.loadmat("/content/gdrive/My Drive/Colab data files/WNvecTrainMicroScaled.mat")
WNvec = mat["ScaledSet"]   # Must be called the same as the variable it was saved 
                    # under in Matlab (or however the block was created)
WNvec = np.array(WNvec)
#WNvec = np.transpose(WNvec)

mat = scipy.io.loadmat("/content/gdrive/My Drive/Colab data files/RTSvecTrainMicroScaled.mat")
RTSvec = mat["RTSdata"]   # Must be called the same as the variable it was saved 
                    # under in Matlab (or however the block was created)
RTSvec = np.array(RTSvec)
'''
        Plot a few to double check you're working with the correct data
'''
'''
x = np.arange(0,1500)
for i in range(9000,10000):
        p = WNvec[i,:]
        plt.plot(x,p)
        plt.savefig('C:/Users/Ben WORK ONLY/Desktop/GH repos/RTS ML detect beta/%d' % (i))
        plt.close()
'''
'''
        Randomize the order of columns(?) and combine data sets
'''

np.random.shuffle(RTSvec)
np.random.shuffle(WNvec)

RTStrain,RTStest = RTSvec[:80000,:],RTSvec[80000:,:]
WNtrain,WNtest = WNvec[:80000,:],WNvec[80000:,:]


x_train = np.concatenate((RTStrain,WNtrain),axis=0)
x_test = np.concatenate((RTStest,WNtest),axis=0)

y_train = np.zeros(160000)
y_train[80000:]=1
y_test = np.zeros(20000)
y_test[10000:]=1

     
del RTStest
del RTStrain
del RTSvec
del WNtest
del WNtrain
del WNvec

###Inspect the variables

In [4]:
variables = %who_ls
for v in variables:
    print(v, type(eval(v)).__name__, str(eval(v)))


print('x_train = ',np.shape(x_train))
print('x_test = ',np.shape(x_test))
print('y_train = ',np.shape(y_train))
print('y_test = ',np.shape(y_test))


drive module <module 'google.colab.drive' from '/usr/local/lib/python3.6/dist-packages/google/colab/drive.py'>
h5py module <module 'h5py' from '/usr/local/lib/python3.6/dist-packages/h5py/__init__.py'>
mat dict {'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Sun Dec 16 16:40:38 2018', '__version__': '1.0', '__globals__': [], 'RTSdata': array([[0.37675271, 0.29618924, 0.42501315, ..., 0.27329266, 0.75398446,
        0.52338596],
       [0.55165801, 0.40010127, 0.3982244 , ..., 0.25280917, 0.37281618,
        0.83361195],
       [0.36244859, 0.53482267, 0.66167355, ..., 0.73957686, 0.30839801,
        0.31334526],
       ...,
       [0.999001  , 0.6789738 , 0.80460875, ..., 0.93238997, 0.62244717,
        0.73275286],
       [0.24328589, 0.2917906 , 0.24590173, ..., 0.23232643, 0.33148367,
        0.29759837],
       [0.26521908, 0.27953052, 0.22466222, ..., 0.81539527, 0.79022043,
        0.75770402]])}
matplotlib module <module 'matplotlib' from '/usr/local/lib/pyt

### Define the structure of the model



In [6]:
from keras.models import Sequential
from keras.layers import Dense, Dropout,Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D, Flatten, LSTM
import numpy as np

X_train = np.expand_dims(x_train, axis=2) # reshape (569, 30) to (569, 30, 1) 
X_test = np.expand_dims(x_test, axis=2) # reshape (569, 30) to (569, 30, 1) 

model = Sequential()
model.add(Conv1D(32, 12, activation='relu',padding = 'same', input_shape=(1500, 1)))
model.add(MaxPooling1D(3))
model.add(Dropout(0.5))
model.add(Conv1D(64, 12, activation='relu',padding = 'same'))
model.add(MaxPooling1D(3))
model.add(Dropout(0.5))
model.add(Conv1D(128, 12, activation='relu',padding = 'same'))
model.add(GlobalAveragePooling1D())
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

Using TensorFlow backend.


### Fit the model and Score

In [8]:
model.fit(X_train, y_train, batch_size=16, epochs=2)
score = model.evaluate(X_test, y_test, batch_size=16)

Epoch 1/2
160000/160000 [==============================] - 137s 856us/step - loss: 0.2691 - acc: 0.8967
Epoch 2/2
20000/20000 [==============================] - 5s 272us/step


### Model Summary

In [0]:
model.summary()

###Simple Plot of Model

In [0]:
# https://pypi.python.org/pypi/pydot
!apt-get -qq install -y graphviz && pip install -q pydot
import pydot

In [0]:
from keras.utils import plot_model
plot_model(model, to_file='/content/gdrive/My Drive/Colab data files/model_alpha.png')